<a href="https://colab.research.google.com/github/PuHelena/Chatbot/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B9_%D0%B1%D0%BB%D0%BE%D0%BA_%7C_%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE_%D1%81%D0%BE%D1%81%D1%82%D1%8F%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%7C_%D0%94%D0%97_Pro_%7C_%D0%A3%D0%98%D0%98%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. Сделайте ConditionalGAN (Генеративно-состязательная сеть с условием) для генерации изображений Fashion Mnist, но вместо индекса изображения на label подавайте рукописную цифру из Mnist картинкой 28 на 28.
2. Добейтесь максимальной детализации изображений, насколько получится.
3. Попробуйте сгенерировать образ по рукописным цифрам, нарисованным в paint.

In [ ]:
# Загрузка библиотек

# Базовые слои keras
from keras.layers import Input, Dense, Reshape, Flatten, Dropout

# Еще базовые слои
from keras.layers import BatchNormalization, Activation, MaxPooling2D, concatenate, Embedding, multiply, Conv2DTranspose, Concatenate

# Активационная функция
from keras.layers.advanced_activations import LeakyReLU

# Загрузим сверточные слои
from keras.layers.convolutional import UpSampling2D, Conv2D 

# Загрузим форматы моделей
from keras.models import Sequential, Model 

# Загрузим оптимизатор
from tensorflow.keras.optimizers.legacy import Adam

# Загрузим модули с датасетами
from keras.datasets import mnist, cifar10, fashion_mnist

# Библиотека для отрисовки графиков
import matplotlib.pyplot as plt 

# Импортируем библиотеку для работы с системой
import sys

# Numpy массивы
import numpy as np

# Модуль отрисовки изображений
from PIL import Image 

# Модуль для загрузки уже готовых моделей
from tensorflow.keras.models import load_model

# Библиотека для работы с файловой системой виртуальной машины
import os

In [ ]:
# загрузим базу MNIST

(xTrainMnist, yTrainMnist), (xTestMnist, yTestMnist) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# загрузим базу Fashion MNIST

(xTrainMnistF, yTrainMnistF), (xTestMnistF, yTestMnistF) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
xTrainMnist=np.concatenate([xTrainMnist,xTestMnist])
yTrainMnist=np.concatenate([yTrainMnist,yTestMnist])

print(xTrainMnist.shape)
print(yTrainMnist.shape)

plt.imshow(xTrainMnist[0],cmap='gray')

In [ ]:
xTrainMnist = xTrainMnist/ 127.5 - 1.0

xTrainMnist = xTrainMnist.reshape(xTrainMnist.shape[0], 28, 28, 1)

print(xTrainMnist.shape)
print(yTrainMnist.shape)

In [ ]:
xTrainMnistF=np.concatenate([xTrainMnistF,xTestMnistF])
yTrainMnistF=np.concatenate([yTrainMnistF,yTestMnistF])

plt.imshow(xTrainMnistF[0],cmap='gray')

In [ ]:
xTrainMnistF = xTrainMnistF / 127.5 - 1.0

xTrainMnistF = xTrainMnistF.reshape(xTrainMnistF.shape[0], 28, 28, 1) 

print(xTrainMnistF.shape)
print(yTrainMnistF.shape)

In [ ]:
num_classes = len(set(yTrainMnist))

print('Всего классов у нас будет ', num_classes)

print('Fashion MNIST')


for i in range (num_classes):
    temp = np.array(np.where(yTrainMnistF==i))  
    print(temp.shape)

print()
print('MNIST')

for i in range (num_classes):
    temp = np.array(np.where(yTrainMnist==i)) 
    print(temp.shape)

In [ ]:
def createSortedXTrain(trainLen):
  '''
  В данной функции происходит синхронизация количества элементов из разных выборок.

  Args:
    trainLen    - количество элементов, которое должно остаться в каждой базе.

  Returns:
    trainMnist  - обучающая выборка из картинок цифр 
    trainMnistF - обучающая выборка из картинок одежды
  '''
  
  # Создадим пустой список обучающей выборки цифр
  trainMnist = []                                                   
  
  # Создадим пустой список обучающей выборки одежды 
  trainMnistF = []                                                    
  
  # Пройдемся по всем классам
  for i in range(num_classes):
    
    # Если метка совпадает с номером класса, запишем цифру в переменную
    currTrainMnist = xTrainMnist[yTrainMnist==i,:]
    # Если метка совпадает с номером класса, запишем вариант одежды в переменную
    currTrainMnistF = xTrainMnistF[yTrainMnistF.flatten()==i,:]
    # Оставим только ограниченное число элементов цифр (в данном случае 6300)
    currTrainMnist = currTrainMnist[:trainLen]
    # Оставим только ограниченное число элементов одежды (в данном случае 6300)
    currTrainMnistF = currTrainMnistF[:trainLen]
    # Занесем в подготовленный пустой список цифр список полученных элементов текущего класса 
    trainMnist.extend(list(currTrainMnist))
    # Занесем в подготовленный пустой список одежды список полученных элементов текущего класса
    trainMnistF.extend(list(currTrainMnistF))
  
  # Преобразуем списки в Numpy массивы
  trainMnist = np.array(trainMnist)
  trainMnistF = np.array(trainMnistF)
  
  # Функция возвращает обучающие выборк цифр и одежды
  return trainMnist, trainMnistF

In [ ]:
trainMnist, trainMnistF = createSortedXTrain(6300)

print('Размерность выборки цифр ',trainMnist.shape)
print('Размерность выборки одежды ',trainMnistF.shape)

trainMnist = trainMnist.reshape(trainMnist.shape[0], 28, 28, 1)
trainMnist.shape

In [ ]:
def plotImages(startIndex, 
               xTrain, 
               pred):
  '''
  Объявляем функцию, которая будет выводить картинки

  Args:
    startIndex   - начиная с какого индекса будем отображать 
    xTrain       - обучающая выборка (по сути вход сети)
    pred         - результат работы сети  
  
  Функция ничего не возвращает
  '''

  # Сколько картинок хотим показать
  n = 5  
  # Зададим размер этих картинок
  plt.figure(figsize=(10, 4))

  # Пройдемся по номеру каждой картинки
  for i in range(n):
      # Сразу утвердим индекс для картинки, которую рассматриваем
      index = i + startIndex
      # Подготовим поле отображения картинки
      ax = plt.subplot(2, n, i + 1)
      # Покажем картинку под номером index. Предварительно преобразовав ее в 28х28
      plt.imshow(xTrain[index].reshape(28, 28))
      # Сменим палитру
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # Показываем восстановленные картинки
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(pred[index].reshape(28,28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()

plotImages(6298, trainMnist, trainMnistF)

In [ ]:
channels = 1                                                                    # Зададим количество каналов цветности
noise_chanells = 128                                                            # Зададим количество каналов для генерации шума
latent_dim = 100                                                                # Зададим размер скрытого пространства
img_rows,img_cols = 28,28                                                       # Размер строк и столбцов             
img_shape = (img_rows, img_cols, channels)                                      # Общая размерность картинки (28,28,1)
optimizer = Adam(0.0002, 0.5)                                                   # Оптимизатор Adam. Шаг lr = 0.0002. Альфа 0.5
batch_size = 128                                                                # Размер батча определим как 128
print(img_shape)                                                                # Выведем размерность одной картинки  

In [ ]:
def create_conditional_generator():
  '''
  Данная функция создает генератор с условием. В ней имеется два входа. На первый подается шум 
  в виде вектора latent_dim (noise). На второй картинка размером (28,28,1). 

  Аргументов на вход нет.

  Returns:
    generator - модель генератора
  '''

  generator = Sequential()                                                      # Создадим сеть линейной направленности
  generator.add (Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))    # Добавим сверточный слой Conv2DTranspose 
  generator.add (LeakyReLU(alpha=0.2))                                          # Добавим слой LeakyReLU
  generator.add (Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))    # Еще один Conv2DTranspose
  generator.add (LeakyReLU(alpha=0.2))                                          # Еще слой активационной функции
  generator.add (Conv2D (1, (7,7), activation='tanh', padding='same'))          # Сверточный слой, который выдает 1 картинку

  # Первая ветвь генератора начинается здесь. На вход приходит вектор размером latent_dim (просто одномерный вектор). 
  noise = Input(shape=(latent_dim,))
  noise_modify = Dense (noise_chanells * 7 * 7) (noise)                         # Добавим полносвязный слой 
  noise_modify = LeakyReLU (alpha=0.2) (noise_modify)                           # Еще слой активационной функции
  noise_modify = Reshape ((7,7,noise_chanells)) (noise_modify)                  # Преобразуем в картинку (7,7,кол-во каналов)
  
  # Вторая ветвь генератора начинается здесь. На вход приходит картинка 28х28х1 (цифра из MNIST)
  label = Input(shape=(28,28,1))
  label_modify = Flatten() (label)                                              # Этот слой преобразует картинку в длинный вектор
  label_modify = Dense (7*7) (label_modify)                                     # Добавим полносвязный слой размером 7х7=49
  label_modify = Reshape ((7,7,1)) (label_modify)                               # Преобразуем в картинку 7х7х1
  
  model_input = concatenate([noise_modify, label_modify])                       # Объединим выходы в переменную model_input    
  img = generator(model_input)                                                  # Передадим в генератор результат объединения
  
  generator = Model([noise, label], img)                                        # Соберем каркас генератора.
  
  # Возвращаем созданный генератор
  return generator

In [ ]:
def create_conditional_discriminator():
  '''
  Создадим функцию дискриминатора с условием. На вход он будет получать две картинки 28х28х1

  Аргументов функция не получает.

  Returns:
    discriminator - возвращает дискриминатор
  '''

  discriminator = Sequential()                                                  # Создадим сеть линейной направленности 
  discriminator.add(Conv2D(32,(3,3),strides=(2,2), padding='same'))             # Добавим сверточный слой 32 нейрона
  discriminator.add(LeakyReLU(alpha=0.2))                                       # Слой активационной функции
  discriminator.add(Conv2D(32,(3,3),strides=(2,2), padding='same'))             # Еще один сверточный слой
  discriminator.add(LeakyReLU(alpha=0.2))                                       # Слой активационной функции
  discriminator.add(Flatten())                                                  # Выравнивающий слой
  discriminator.add(Dense(32))                                                  # Полносвязный слой на 32 нейрона
  discriminator.add(Dropout(0.4))                                               # Слой для случайного отключения нейронов
  discriminator.add(Dense(1,activation='sigmoid'))                              # Выходом имеется полносвязный слой с одним нейроном
  
  img = Input(shape=img_shape)                                                  # Первый вход - картинка из базы одежды
  label = Input(shape=(28,28,1))                                                # Второй вход - картинка из базы цифр
  model_input = concatenate([img,label])                                        # Объединяем данные от двух входов
  validity = discriminator(model_input)                                         # Записываем вход в дискриминатор
  
  # Собираем "скелет" дискриминатора из двух входов (img,label)  и одного выхода (validity)
  discriminator = Model([img, label], validity)
  
  # Компилируем сеть
  discriminator.compile(loss='binary_crossentropy',         # Бинарную кроссэнтропию выбрали так как у нас два состояния
              optimizer=optimizer,                          # Выбрали оптимизатор
              metrics=['accuracy'])                         # Проставили метрику  
 
  # Застопорим обучение дискриминатора, так как он должен обучаться отдельно от генератора
  discriminator.trainable = False
  
  # Функция возвращает дискриминатор
  return discriminator

In [ ]:
yTrain=[]                                                                       # Создадим список

# Пройдемся по всем 10 классам
for i in range(10):  
  yTrain.extend([i for n in range(6300)])                                       # Запишем метки в соответствующие классы

yTrain=np.array(yTrain)                       

In [ ]:
def sample_images_conditional(epoch, 
                              gen):
  '''
  Функция будет отображать картинки, созданные генератором.

  Arguments:
     epoch   - номер эпохи
     gen     - передаваемый генератор
  
  Функция ничего не возвращает. Только выводит картинки.
  '''
  
  # Определим количество строк и колонок 
  r, c = 2, 5                                                                    
  
  # Создается матрица значений нормального распределения
  noise = np.random.normal(0, 1, (r * c, latent_dim))                           
  
  # Создадим пустой список
  sampled_labels=[]                                                             

  # Генерируем массив из 10 случайных меток (от 0 до 9)
  for i in range(num_classes):                                                  
    # Берём случайную метку из всех меток i-го класса
    sampled_labels.append(trainMnist[yTrain==i][np.random.randint(0,trainMnist[yTrain==i].shape[0])])
  
  # Преобразование списка меток в массив
  sampled_labels = np.array(sampled_labels)
  
  # Генерация картинок. На вход генератора подается noise и список меток
  gen_imgs = gen.predict([noise, sampled_labels])

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  # Подготовим "полотна" для отрисовки картинок
  fig, axs = plt.subplots(r, c)

  # Создаем счетчик картинок
  cnt = 0

  # Пройдемся по рядам картинок
  for i in range(r):
      
      # Пройдемся по колонкам картинок
      for j in range(c):
          # Отобразим картинку под номером [i,j], преобразовав полученную из генератора в (28,28)
          axs[i,j].imshow(gen_imgs[cnt].reshape((28,28)), cmap='gray')
          # Оси не показываем
          axs[i,j].axis('off')
          # После отображения картинки счетчик увеличим на 1
          cnt += 1

  # Отобразим результаты
  plt.show()
  
  # Закроем поток
  plt.close()

In [ ]:
def train_conditional(generator, 
                      discriminator, 
                      epochs=30000, 
                      batch_size=128, 
                      sample_interval = 200):
  '''
  Данная функция проводит обучение сети. На вход принимает генератор и дискриминатор. А также количество эпох,
  размер батча, интервал сэмплирования.
  
  Args:
    generator       - подаем на вход генератор
    discriminator   - подаем на вход дискриминатор
    epochs          - количество эпох
    batch_size      - размер батча
    sample_interval - интервал отображения картинок

  Функция ничего не возвращает
  '''

  # Создадим массив единичек размером batch_size (вертикальный)
  valid = np.ones((batch_size, 1))

  # Создадим массив нулей размером batch_size (вертикальный)  
  fake = np.zeros((batch_size, 1))

  # Создадим входной слой размером latent_dim.   
  noise = Input(shape=(latent_dim,))
   # Создадим входной слой размером (28,28,1)
  label = Input(shape=img_shape)

  # Скомпонуем генератор и подадим ему на вход два наших слоя
  img = generator([noise, label])

  # Теперь скомпонуем  дискриминатор и подадим ему на вход один из слоев и результат работы генератора.  
  val = discriminator([img, label])
  
  # Соберем полностью модель сети из двух входов (noise,label) и одного выхода (val)
  combined = Model([noise, label], val)

  # Компилируем сеть используя бинарную кроссэнтропию и оптимизатор Adam (выше мы его задавали).
  combined.compile(loss=['binary_crossentropy'],
      optimizer=optimizer)

  # Создадим набор пустых списков для ошибок дискриминатора, генератора и точности 
  d_loss_list = []
  g_loss_list = []
  d_acc_list = []
  
  # Запускаем цикл эпох обучения
  for epoch in range(epochs):
    # Выберем случайно массив индексов от 0 до 63000 размером batch_size    
    idx1 = np.random.randint(0, trainMnistF.shape[0], batch_size) 

    # Загрузим картинки с выбранными индексами в imgs
    imgs = trainMnistF[idx1]

    # А цифры с оответствующими индексами в labels
    labels = trainMnist[idx1]
    # Создается матрица размером (batch_size,latent_dim) из значений нормального распределения
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Создаем несколько новых картинок
    gen_imgs = generator.predict([noise, labels])

    # Обучение дискриминатора на одном батче с правильными картинками
    d_loss_real = discriminator.train_on_batch([imgs, labels], valid)

    # Обучение дискриминатора на одном батче с фейковыми картинками
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)

    # Создание суммарного лосса дискриминатора (разделив пополам чтобы получить значения от 0 до 1)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Возьмем некоторый набор индексов размером с batch_size
    idx2 = np.random.randint(0, trainMnist.shape[0], batch_size)

    # Запишем картинки, связанные с этими индексами в отдельную переменную
    sampled_labels = trainMnist[idx2]

    # Тренируем сеть на одном батче картинок. 
    g_loss = combined.train_on_batch([noise, sampled_labels], valid)

    # Запишем прогресс на данной эпохе обучения
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    
    # Запишем в соответствующий список очередную ошибку дискриминатора
    d_loss_list.append(d_loss[0])

    # Запишем в соответствующий список очередную ошибку генератора
    g_loss_list.append(g_loss)
    
    # Запишем в список точность дискриминатора
    d_acc_list.append(100*d_loss[1])

    # Если дошли до 200-ой эпохи или до предпоследней, то
    if ((epoch % sample_interval == 0) | (epoch == epochs-1)):
        # покажем очередную порцию картинок. Получив при этом номер текущей эпохи и генератор
        sample_images_conditional(epoch, generator)
        
  # Выводим графики ошибок дискриминатора и генератора, их легенды.
  plt.plot(d_loss_list, label="Ошибка дискриминатора")
  plt.plot(g_loss_list, label="Ошибка генератора")
  plt.legend()
  plt.show()
  # Также выведем график точности распознавания дискриминатора и легенду.
  plt.plot(d_acc_list, label="Точность распознавания дискриминатора")
  plt.legend()
  plt.show()
    

In [ ]:
conditional_generator = create_conditional_generator()
conditional_discriminator = create_conditional_discriminator()

train_conditional(conditional_generator, conditional_discriminator, 2000, sample_interval=200)

In [ ]:
# Скомпилируем обученный генератор.
conditional_generator.compile()

# И сохраним его в файл .h5 .
conditional_generator.save('cond_gen_1.h5')
# Загрузим обученную модель генератора.
test_gen = load_model('cond_gen_1.h5')
# Протестируем его работу. 
sample_images_conditional(0,test_gen)

In [ ]:
train_conditional(conditional_generator, conditional_discriminator, 1000, sample_interval=200)

In [ ]:
# Скомпилируем обученный генератор.
conditional_generator.compile()

# И сохраним его в файл .h5 .
conditional_generator.save('cond_gen_2.h5')
# Загрузим обученную модель генератора.
test_gen = load_model('cond_gen_2.h5')
# Протестируем его работу. 
sample_images_conditional(0,test_gen)

In [ ]:
def plot_label(indexes, 
               gen, 
               isHandWritten = False):
  '''
  Функция предоставляет функционал для работы с обученным генератором. Ее задача отобразить картинки.

  Args:
    indexes       - индексы по которым начинаем брать картинки цифр
    gen           - сюда подаем наш собранный и обученный генератор
    isHandWritten - эта переменная отвечает за выбор свои рукописные цифры берем или из базы MNIST

  Функция ничего не возвращает 
  '''
  sampled_labels = []                            # Создадим пустой список
  paint_pictures = []                            # Создадим пустой список для самодельных картинок
  pictures_path  = '/content/digits/'            # Путь к картинкам
  dir_pictures  = os.listdir('/content/digits/') # Список картинок в директории

  # Пройдемся по директории с картинками
  for i in range(len(dir_pictures)):
    
    # Заполним список рукописными картинками
    paint_pictures.append(image.load_img(pictures_path +                        
                                         random.choice(dir_pictures), # Случайный выбор
                                         target_size = (28, 28),      # Размер 28х28  
                                         color_mode = "grayscale"))   # Цветовая палитра - серая

  # Определимся откуда будем получать картинки 
  if (isHandWritten):  
       
    # Определим сколько картинок
    length = len(paint_pictures)                                                

    # Пройдемся по всем картинкам из списка
    for i in range(length):    
      
      # Преобразуем картинку в массив
      sampled_labels.append(image.img_to_array(paint_pictures[i]))            
      # Нормируем
      sampled_labels[i] = 255-sampled_labels[i]                                 
      # Преобразуем значения массива в числа от -1 до 1
      sampled_labels[i] = sampled_labels[i]/127.5 - 1.0                         
      # Преобразуем к размерности 28,28,1
      sampled_labels[i] = sampled_labels[i].reshape(28,28, 1)                    

  # Если мы решили брать метки из MNIST
  
  else:
    # Получим количество индексов
    length = len(indexes)                                                       

    # Проходимся по всем индексам
    for i in indexes:
      # Запишем в список цифры, полученные из обучающей выборки, по метке индекса
      sampled_labels.append(trainMnist[yTrain==i][np.random.randint(0,trainMnist[yTrain==i].shape[0])])  
  
  # Преобразуем список меток в массив
  sampled_labels = np.array(sampled_labels)                                     
  
  # Создадим массив шума
  noise = np.random.normal(0, 1, (length, latent_dim))                           
  
  # Подадим шумы и цифры в генератор
  gen_imgs = gen.predict([noise, sampled_labels])                               
  
  # Усредним показания генератора
  gen_imgs = 0.5 * gen_imgs + 0.5                                               
  
  # Подготовим "полотна" для картинок
  fig, axs = plt.subplots(2,length)                                             

  # Если количество индексов равно 1
  if (length==1):
    # Решейпим "полотна" для 2 рядов и 1 колонки
    axs = axs.reshape(2,1)                                                      
  
  # Счетчик в 0
  cnt = 0                                                                       

  # Пройдемся по всем индексам (их у нас всего length)
  for i in range(length):      
    
    # Отобразим сгенерированную картинку
    axs[1,i].imshow(gen_imgs[cnt].reshape((28,28)), cmap='gray')                
    # Не показываем оси
    axs[1,i].axis('off')                                                        
    # Отобразим цифру
    axs[0,i].imshow(sampled_labels[cnt].reshape((28,28)), cmap='gray')          
    # Не показываем оси
    axs[0,i].axis('off')                                                        
    # Счетчик увеличим на 1
    cnt += 1                                                                    
  
  # Отобразим
  plt.show()  

In [ ]:
# Этой переменной задается количество картинок
count = 3                          

# Выберем случайные индексы
idx = np.random.randint(0, num_classes, count)  

# Выведем результат работы 
print ("Случайный массив: ", idx)
print ("Соответствующие классы fashion_mnist: ")

plot_label(idx, conditional_generator, False)           